In [ ]:
#IMPORTANT POINTS TO REMEMBER WHILE RUNNING THIS CODE
# 1. Specify the folder path where all the files are present
# 2. Specify the starting year and the window size 
# 3. The code is written sector wise -- for this we will need (for_puspendra.sas7bdat) to be kept in the pwd
# 4. Before running code make sure all the files in your folder are renamed as AccessionNumber.txt
# 5. If you want to use the words from particular financial books -- specify the texts book and their page number 
# 6. If you want to use the words from particular financial books -- tune the variable to onlyWords to 1

In [ ]:
#Libraries to load
import os
import re
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from pprint import pprint
import pandas as pd
import numpy as np
import datetime
import datetime
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import nltk

In [ ]:
from gensim.corpora import Dictionary
#load library
import os
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import gensim
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models import Phrases

In [ ]:
import numpy as np
import pandas as pd
np.random.seed(400)
stemmer = SnowballStemmer("english")

In [ ]:
# Extracting all the sectors
st = pd.read_sas('for_puspendra.sas7bdat')
st['GSECTOR'] = st['GSECTOR'].astype(str)
st['GSECTOR'] =st['GSECTOR'].str.strip("b'")
st['accession'] = st['accession'].astype(str)
st['accession'] =st['accession'].str.strip("b'")
st['year'] = st['accession'].str.split('-').str[1]
st['year'] = st['year'].astype(int)
gsectors=st['GSECTOR'].unique().tolist()
gsectors=gsectors[0:11]

In [ ]:
# Use this to download libraries if running code for the first time
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [ ]:
#function to preprocess the textual data
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def preprocess_text(text):
    text = text.lower()
    # Remove auto-generated electronic filings
    text = re.sub(r'<TYPE>.*?</TYPE>', '', text, flags=re.DOTALL)
    
    text = re.sub(r'<.*?>', '', text)
    
    # Remove XBRL data
    text = re.sub(r'<XBRL>.*?</XBRL>', '', text, flags=re.DOTALL)

    # Remove outlier text after </TEXT> tag
    text = re.sub(r'</TEXT>.*', '', text, flags=re.DOTALL)

    # Remove paragraph tags
    text = re.sub(r'<p>', '', text)

    # Remove punctuation marks except periods and commas
    text = re.sub(r'[^\w\s.,]', '', text)

    # Remove text between <TABLE> and </TABLE> tags
    text = re.sub(r'<TABLE>.*?</TABLE>', '', text, flags=re.DOTALL)

    # Remove paragraphs with less than 50 words
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if len(p.split()) >= 20]
    text = '\n'.join(paragraphs)

    # Clean up extra spaces
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    text = text.lower()
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\d+', '', text)
    
    # Remove dates and years
    text = re.sub(r'\b\d{1,2}/\d{1,2}/\d{2,4}\b', '', text)  # Removing dates in the format MM/DD/YYYY or MM/DD/YY
    text = re.sub(r'\b\d{1,2}-\d{1,2}-\d{2,4}\b', '', text)  # Removing dates in the format MM-DD-YYYY or MM-DD-YY
    text = re.sub(r'\b\d{1,2}\s+\w+\s+\d{2,4}\b', '', text)  # Removing dates in the format DD Month YYYY or DD Month YY
    text = re.sub(r'\b\d{4}\b', '', text)  # Removing standalone years (4 digits)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    polite_words = [
        'please', 'thank', 'thank you', 'excuse', 'pardon', 'may I', ' please', 'appreciate', 
        "grateful", ' don\'t mind', 'sorry', 'inconvenience', ' welcome', 'apologies', 'kindly',
        ' mind', ' bother', 'allow ', 'possible', 'trouble','this'
    ]
    stop_words.update(polite_words)

    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    text = ' '.join(tokens)

    return text


In [ ]:
# Function to create bigrams of word from the textual data
from gensim.utils import simple_preprocess
def preprocess_documents(documents):
    processed_docs = [simple_preprocess(doc, deacc=True, min_len=2, max_len=15) for doc in documents]
#     processed_docs = [word_tokenize(doc) for doc in documents]
    processed_docs = [list(map(str.encode, doc)) for doc in processed_docs]
    
    # Create bigrams
    bigram_mod = []
    for doc in processed_docs:
        bigrams = list(ngrams(doc, 2))
        bigram_mod.append([' '.join(map(bytes.decode, bigram)) for bigram in bigrams])

    return bigram_mod

In [ ]:
import PyPDF2  # Import the PyPDF2 library for PDF operations

filename = 'FinancialDictionary.pdf'  # Specify the PDF file name

pdfFileObj = open(filename, 'rb')  # Open the PDF file in read-binary mode
pdfReader = PyPDF2.PdfReader(pdfFileObj)  # Create a PdfReader object to read the PDF
num_pages = len(pdfReader.pages)  # Get the total number of pages in the PDF

start_page = 1  # Specify the starting page
end_page = 5    # Specify the ending page

count = start_page - 1  # Initialize the count variable to the starting page index
text = ""  # Initialize an empty string to store the extracted text

# Iterate over the desired pages and extract text from each page
while count < num_pages and count < end_page:
    pageObj = pdfReader.pages[count]  # Get the page object for the current page
    count += 1  # Increment the count variable
    text += pageObj.extract_text()  # Extract text from the page and append it to the 'text' string

# Check if text is empty after processing desired pages
if text != "":
    text = text  # If text is not empty, assign it to the 'text' variable
else:
    # If text is empty, extract text using textract library from the specified URL
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

text = text.encode('ascii', 'ignore').lower()  # Convert the text to lowercase and remove non-ASCII characters

uniqueWords = [] 
for i in keywords:
      if not i in uniqueWords:
            uniqueWords.append(i);
                
uniqueWords
wordss=uniqueWords

In [ ]:
import os
import re
import pandas as pd
import csv

folder_path = 'ksen'  # Folder path containing the 10K files
csv_file = 'cleaned_text_10K.csv'  # CSV file path to store the cleaned text
num_of_topics = 30 #Specify the number of topics
yr = 5 #Specify the starting year
window = 3 #Specify the window size
onlyWords=0 # 1 for yes and 0 for no
# Iterate over the desired years
for x in range(yr, yr + 22 - 5 - window + 1, 1):
#     if x == 15:
#         break
        
    # Filter the DataFrame based on the year and sector
    if window == 1:
        filtered_df = st[(st['year'].isin([x]))]
    if window == 3:
        filtered_df = st[(st['year'].isin([x, x + 1, x + 2]))]
    if window == 5:
        filtered_df = st[(st['year'].isin([x, x + 1, x + 2, x + 3, x + 4]))]

    # Get the accession numbers from the filtered DataFrame
    accession_numbers = filtered_df['accession'].tolist()

    # Open the CSV file for writing
    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Accession Number', 'Cleaned Text'])

        # Iterate over the accession numbers
        for accession_number in accession_numbers:
            filename = accession_number + '.txt'
            file_path = os.path.join(folder_path, filename)

            if os.path.isfile(file_path):  # Check if the path is a file
                with open(file_path, 'r') as file:
                    content = file.read()

                # Preprocess and clean the text
                cleaned_text = preprocess_text(content)

                if not cleaned_text:
                    cleaned_text = content

                # Write the cleaned text and accession number to the CSV file
                writer.writerow([accession_number, cleaned_text])
                    
        # Read the cleaned text DataFrame
        df = pd.read_csv('cleaned_text_10K.csv')
        df = df.reset_index(drop=True)
        
        if(onlyWords):
            df['Cleaned Text']=df['Cleaned Text'].str.lower().str.split()
            df['Cleaned Text'] = df['Cleaned Text'].apply(lambda words: [word for word in words if word in wordss])
            df['Cleaned Text'] = df['Cleaned Text'].apply(lambda x: ' '.join(x))
        
        # Convert the 'Cleaned Text' column to a list
        flat_list = df['Cleaned Text'].tolist()
        
        # Preprocess the documents
        processed_docs = preprocess_documents(flat_list)
        
        # Create a Gensim Dictionary object
        dictionary = gensim.corpora.Dictionary(processed_docs)

        # Remove extreme words (very common and very rare) from the dictionary
        dictionary.filter_extremes(no_below=2, no_above=0.01)
        
        # Check if the dictionary is empty after filtering
        if len(dictionary) == 0:
            dictionary = gensim.corpora.Dictionary(processed_docs)  

        # Create bag-of-word (BoW) models for each document
        bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    
        # Create TF-IDF model from the BoW corpus
        tfidf = models.TfidfModel(bow_corpus)
        corpus_tfidf = tfidf[bow_corpus]

        # Apply LDA topic modeling using TF-IDF
        lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                                     num_topics=num_of_topics, 
                                                     id2word=dictionary, 
                                                     passes=10, 
                                                     workers=4)
        
        # Define the output filename for the topics
        if(window==3):
            filenameee = f"topics_{int(x) + 2000}_{int(x + 1) + 2000}_{int(x + 2) + 2000}.csv"

        if(window==5):
            filenameee = f"topics_{int(x) + 2000}_{int(x + 1) + 2000}_{int(x + 2) + 2000}_{int(x + 3) + 2000}_{int(x + 4) + 2000}.csv"
        
        if(window==1):
            filenameee = f"topics_{int(x) + 2000}.csv"

        # Write the topics to a CSV file
        with open(filenameee, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Words'])

            for idx, topic in lda_model_tfidf.print_topics(-1):
                topic_parts = topic.split('"')[1::2]  # Extract the words within double quotes
                words = ', '.join(topic_parts)
                writer.writerow([words])

        # Create a DataFrame to store the document-topic distribution
        topic_columns = ["topic" + str(i) for i in range(1, num_of_topics + 1)]
        documents_lda_tfidf_topics = pd.DataFrame(columns=topic_columns)

        # Iterate over the documents
        for i in range(len(bow_corpus)):
            if i % 500 == 0:
                print(i)
            
            # Initialize the topic distribution for the document
            documents_lda_tfidf_topics.loc[i] = np.zeros(num_of_topics)
            
            # Get the topic distribution for the document
            output_tfidf = lda_model_tfidf.get_document_topics(bow_corpus[i])
            
            # Update the document-topic distribution DataFrame
            for k in range(len(output_tfidf)):
                a = output_tfidf[k][0]
                b = output_tfidf[k][1]
                documents_lda_tfidf_topics.iloc[i, a] = b

        # Filter the DataFrame based on the accession numbers for the next year
        if(window==3):
            if (x + 2) < 10 and x > 5:
                filtered_df = df[df['Accession Number'].str.contains('-0' + str(x + 2) + '-')]
            elif x == 5:   
                filtered_df = df[df['Accession Number'].str.contains('-0' + str(x + 2)) |
                              df['Accession Number'].str.contains('-0' + str(x + 1)) |
                              df['Accession Number'].str.contains('-0' + str(x))]
            else:
                filtered_df = df[df['Accession Number'].str.contains('-' + str(x + 2) + '-')]

        if(window==5):
            if (x + 4) >= 10:
                filtered_df = df[df['Accession Number'].str.contains('-' + str(x + 4) + '-')]
            elif x == 5:   
                filtered_df = df[df['Accession Number'].str.contains('-0' + str(x + 2)) |
                              df['Accession Number'].str.contains('-0' + str(x + 1)) |
                              df['Accession Number'].str.contains('-0' + str(x))|
                              df['Accession Number'].str.contains('-0' + str(x + 3))|
                              df['Accession Number'].str.contains('-0' + str(x + 4))]
        
        if(window==1):
            if(x>9):
                filtered_df = df[df['Accession Number'].str.contains('-' + str(x) + '-')]
            else:
                filtered_df = df[df['Accession Number'].str.contains('-0' + str(x) + '-')]



        # Concatenate the filtered DataFrame with the document-topic distribution DataFrame
        documents_lda_tfidf_topics = pd.concat([filtered_df[['Accession Number']], documents_lda_tfidf_topics], axis=1)
        
        # Drop any rows with missing values
        documents_lda_tfidf_topics = documents_lda_tfidf_topics.dropna()
        
        # Reset the index of the DataFrame
        documents_lda_tfidf_topics = documents_lda_tfidf_topics.reset_index(drop=True)
        
        # Define the output filename for the document-topic distribution
        if(window==1):
            filenamee = f"TopicModellingText_TFIDF_{int(x) + 2000}.csv"
        if(window==3):  
            filenamee = f"TopicModellingText_TFIDF_{int(x + 2) + 2000}.csv"
        if(window==5):  
            filenamee = f"TopicModellingText_TFIDF_{int(x + 4) + 2000}.csv"  

        # Save the DataFrame to a CSV file
        documents_lda_tfidf_topics.to_csv(filenamee)
